## Тренировочный тест: Доверительные интервалы для среднего

### Вопрос 1.
Пусть $X\sim F(x)$ — случайная величина с произвольным распределением, неизвестным математическим ожиданием $\mathbb{E}X$ и известной дисперсией $\mathbb{D}X$. Как выглядит доверительный интервал для $\mathbb{E}X$ с приближённым уровнем доверия $100(1-\alpha)\%$?
- $\bar X_n \pm z_{1-\frac \alpha 2} \sqrt{\frac {\mathbb{D}X}{n}}$

### Вопрос 2.
Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на миллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

**Будьте осторожны при использовании метода `std()`!** Дело в том, что у объекта `numpy` он по умолчанию вычисляется как $\sqrt {\frac {1}{n}\sum\limits_{i=1}^{n}(X_i−\bar X)^2}$,

а у объекта `pandas` — как $\sqrt {\frac {1}{n-1}\sum\limits_{i=1}^{n}(X_i−\bar X)^2}$.

Нас интересует только второй вариант, несмещённая оценка стандартного отклонения.

Чтобы не думать всё время о том, правильно ли вычисляется в вашем случае `std()`, можно всегда использовать `std(ddof=1) (ddof — difference in degrees of freedom)`, тогда нормировка всегда будет на `n-1`.


In [1]:
import pandas as pd

data = pd.read_csv('water.txt', delimiter = '\t')

In [2]:
data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [3]:
from numpy import sqrt
from statsmodels.stats.weightstats import _tconfint_generic, _zconfint_generic
mean_std = data['mortality'].std(ddof=1)/sqrt(61)
print "ridge model mean auc 95%% confidence interval", _tconfint_generic(data['mortality'].mean(), mean_std,
                                                                         len(data) - 1,
                                                                         0.05, 'two-sided')

ridge model mean auc 95%% confidence interval (1476.0833413552848, 1572.2117406119285)


### Вопрос 3.
На данных из предыдущего вопроса постройте 95% доверительный интервал для средней годовой смертности по всем южным городам. Чему равна его верхняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [5]:
import numpy as np
_tconfint_generic(data[data.location == 'South'].mortality.mean(),
                  data[data.location == 'South'].mortality.std(ddof=1) / np.sqrt(len(data[data.location == 'South'])),
                  len(data[data.location == 'South']) - 1, 0.05, 'two-sided')

(1320.1517462936238, 1433.463638321761)

### Вопрос 4.
На тех же данных постройте 95% доверительный интервал для средней годовой смертности по всем северным городам. Пересекается ли этот интервал с предыдущим? Как вы думаете, какой из этого можно сделать вывод?
- Интервалы не пересекаются; видимо, средняя смертность на севере и на юге существенно разная

### Вопрос 5.
Пересекаются ли 95% доверительные интервалы для средней жёсткости воды в северных и южных городах?
- Не пересекаются

In [8]:
data_ = data[data.location == 'South']
mean_std_ = data_['hardness'].std(ddof=1)/sqrt(len(data_))
print "95%% confidence interval", _tconfint_generic(data_['hardness'].mean(), mean_std_,
                                                                         len(data_) - 1,
                                                                         0.05, 'two-sided')

95%% confidence interval (53.467198692036106, 86.071262846425441)


In [9]:
data_ = data[data.location == 'North']
mean_std_ = data_['hardness'].std(ddof=1)/sqrt(len(data_))
print "95%% confidence interval", _tconfint_generic(data_['hardness'].mean(), mean_std_,
                                                                         len(data_) - 1,
                                                                         0.05, 'two-sided')

95%% confidence interval (21.422487285724259, 39.377512714275738)


### Вопрос 6.
Вспомним формулу доверительного интервала для среднего нормально распределённой случайной величины с дисперсией $\sigma^2$:

$$\bar{X}_n \pm z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}}$$

При $\sigma=1$ какой нужен объём выборки, чтобы на уровне доверия 95% оценить среднее с точностью $\pm 0.1$?

In [12]:
from scipy import stats
np.ceil((stats.norm.ppf(1-0.05/2) / 0.1)**2)

385.0

### Вопрос 7.
Объём выборки, необходимый для построения доверительного интервала заданной ширины:
- уменьшается с ростом $\alpha$
- увеличивается с уменьшением требуемой ширины
- увеличивается с ростом дисперсии выборки